# Reading website using function calling and jinai

Here's the link if you want to test: https://colab.research.google.com/drive/1vOzO0-wl8NTNvi0VTq5GU8HmsGVpw-sb?usp=sharing

In [1]:
!pip install openai

In [49]:
import requests

url = 'https://maxflowtech.com/'
headers = {
    'Authorization': 'Bearer jina_key_here'
}

response = requests.get(url, headers=headers, verify=False)
print(response.text)


<!DOCTYPE html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta content="width=device-width, initial-scale=1.0" name="viewport">
  <meta name="google-site-verification" content="HSLfOjYZNTfyI68P66UhtO-VohEEAJqoYRcxS96UlJI" />
  <title>Maxflow Technology Inc.</title>
  <meta content="Maxflow Technology Inc. home page. Maxflow provides workflow automation, or process automation software" name="description">
  <meta content="Maxflow AI, workflow automation, GenAI, chat bot, chat interface, process automation, low-code, optimizer, digital twin, tradespace exploration, design optimization" name="keywords">

  <!-- Favicons -->
  <link href="assets/img/favicon.png" rel="icon">
  <link href="assets/img/apple-touch-icon.png" rel="apple-touch-icon">

  <!-- Google Fonts -->
  <link
    href="https://fonts.googleapis.com/css?family=Open+Sans:300,300i,400,400i,600,600i,700,700i|Roboto:300,300i,400,400i,500,500i,600,600i,700,700i|Poppins:300,300i,400,400i,500,500i,600,600i,700,700i"
    

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'maxflowtech.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [50]:
def get_current_weather(location: str, unit: str):
    """Get the current weather in a given location"""
    # Hardcoded response for demo purposes
    return "Trời rét vãi, 0 độ C"

def get_stock_price(symbol: str):
    """get_stock_price"""
    # Không làm gì cả, để hàm trống
    pass

def view_website(url: str) -> str:
    """Fetch and return markdown content from the given URL using JinaAI"""
    jina_url = f'https://r.jina.ai/{url}'
    headers = {
        'Authorization': 'Bearer jina_key_here'
    }

    response = requests.get(jina_url, headers=headers, verify=False)
    if response.status_code == 200:
        return response.text
    else:
        return f"Error: Unable to fetch content from {url} (Status code: {response.status_code})"

In [51]:
from pydantic import TypeAdapter
import inspect

# Introspect the function signatures and create tools dynamically
tools = []

for func in [get_current_weather, get_stock_price, view_website]:
    doc = inspect.getdoc(func)
    type_adapter = TypeAdapter(func).json_schema()

    tool = {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": doc,
            "parameters": type_adapter
        }
    }
    tools.append(tool)

# The generated tools now have proper type validation and descriptions
print(tools)


[{'type': 'function', 'function': {'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'additionalProperties': False, 'properties': {'location': {'title': 'Location', 'type': 'string'}, 'unit': {'title': 'Unit', 'type': 'string'}}, 'required': ['location', 'unit'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'get_stock_price', 'description': 'get_stock_price', 'parameters': {'additionalProperties': False, 'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'type': 'object'}}}, {'type': 'function', 'function': {'name': 'view_website', 'description': 'Fetch and return markdown content from the given URL using JinaAI', 'parameters': {'additionalProperties': False, 'properties': {'url': {'title': 'Url', 'type': 'string'}}, 'required': ['url'], 'type': 'object'}}}]


In [63]:
from pprint import pprint
from openai import OpenAI

# https://platform.openai.com/api-keys
client = OpenAI(
    api_key='key-here',
)
COMPLETION_MODEL = "gpt-4o-mini"

messages = [{"role": "user", "content": "Hãy giúp tôi tìm hiểu về doanh nghiệp Maxflow qua trang web: https://maxflowtech.com/ , giới thiệu chi tiết về công ty và cho tôi một số câu hỏi phỏng vấn dựa vào web đó kèm với câu trả lời"}]

print("Bước 1: Gửi message lên cho LLM")
pprint(messages)

response = client.chat.completions.create(
    model=COMPLETION_MODEL,
    messages=messages,
    tools=tools
)

print("Bước 2: LLM đọc và phân tích ngữ cảnh LLM")
pprint(response)

Bước 1: Gửi message lên cho LLM
[{'content': 'Hãy giúp tôi tìm hiểu về doanh nghiệp Maxflow qua trang web: '
             'https://maxflowtech.com/ , giới thiệu chi tiết về công ty và cho '
             'tôi một số câu hỏi phỏng vấn dựa vào web đó kèm với câu trả lời',
  'role': 'user'}]
Bước 2: LLM đọc và phân tích ngữ cảnh LLM
ChatCompletion(id='chatcmpl-B6vk7Mm3RmQNBGs0GLLWpSkfekHUC', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8yMBcKug0SK36OlDwSoh6SBW', function=Function(arguments='{"url":"https://maxflowtech.com/"}', name='view_website'), type='function')]))], created=1740991631, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_06737a9306', usage=CompletionUsage(completion_tokens=22, prompt_tokens=161, total_tokens=183, completion_tokens_details=

In [64]:
print("Bước 3: Lấy kết quả từ LLM")
tool_call = response.choices[0].message.tool_calls[0]

pprint(tool_call)

Bước 3: Lấy kết quả từ LLM
ChatCompletionMessageToolCall(id='call_8yMBcKug0SK36OlDwSoh6SBW', function=Function(arguments='{"url":"https://maxflowtech.com/"}', name='view_website'), type='function')


In [65]:
import json
print("Bước 4: Chạy function view_website ở máy mình")

# Vì ở đây ta có 3 hàm nên phải check theo name để gọi đúng hàm `view_website`
if tool_call.function.name == 'view_website':
    arguments = json.loads(tool_call.function.arguments)
    view_website_result = view_website(
        arguments.get('url'))
    # Hoặc code này cũng tương tự
    # view_website_result = view_website(**arguments)
    print(f"Kết quả bước 4: {view_website_result}")

Bước 4: Chạy function view_website ở máy mình


/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'r.jina.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Kết quả bước 4: Title: Maxflow Technology Inc.

URL Source: https://maxflowtech.com/

Markdown Content:
### Enterprise Chat Interface

[![Image 1: MaxGPT Enterprise Automation](https://maxflowtech.com/assets/img/maxgpt.png)](https://youtu.be/b1eFx2TRL2I)

### MaxGPT Enterprise Automation with GenAI

MaxGPT is a platform to leveraging GenAI for enterprise automation. It combines workflow automation and GenAI so that users can quickly build and deploy AI Assistants at scale while smoothly transition from existing enterprise sofware to GenAI-empowered eco-system.

### Enterprise Workflow Builder

[![Image 2: About](https://maxflowtech.com/assets/img/about.png)](https://maxflowtech.com/assets/img/about.png)

### Overview

[Maxflow.app](https://maxflow.app/) is a web-based platform for process automation. It allows users to connect simple components together to make workflows that perform flexible and complex jobs. Many components in Maxflow are wrappers of legacy desktop/web apps such as E

In [66]:
print("Bước 5: Gửi kết quả lên cho LLM")
messages.append(response.choices[0].message)
messages.append({
    "role": "tool",
    "tool_call_id": tool_call.id,
    "name": "view_website",
    "content": view_website_result
})

pprint(messages)
final_response = client.chat.completions.create(
    model=COMPLETION_MODEL,
    messages=messages
    # Ở đây không có tools cũng không sao, vì ta không cần gọi nữa
)
print(f"Kết quả cuối cùng từ LLM: {final_response.choices[0].message.content}.")

Bước 5: Gửi kết quả lên cho LLM
[{'content': 'Hãy giúp tôi tìm hiểu về doanh nghiệp Maxflow qua trang web: '
             'https://maxflowtech.com/ , giới thiệu chi tiết về công ty và cho '
             'tôi một số câu hỏi phỏng vấn dựa vào web đó kèm với câu trả lời',
  'role': 'user'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_8yMBcKug0SK36OlDwSoh6SBW', function=Function(arguments='{"url":"https://maxflowtech.com/"}', name='view_website'), type='function')]),
 {'content': 'Title: Maxflow Technology Inc.\n'
             '\n'
             'URL Source: https://maxflowtech.com/\n'
             '\n'
             'Markdown Content:\n'
             '### Enterprise Chat Interface\n'
             '\n'
             '[![Image 1: MaxGPT Enterprise '
             'Automation](https://maxflowtech.com/assets/img/maxgpt.png)](https://youtu.be/b1eFx2TRL2I)\n'
             '\n'
             '#